In [1]:
import time
import numpy as np
import matplotlib.pyplot as plt
import dgl
import networkx as nx
import torch
import torch.nn as nn

import utils
from gsp_utils.baselines_archs import GCNN_2L
from gsp_utils.baselines_modesl import NodeClassModel, GF_NodeClassModel
from gsp_utils.data import normalize_gso
from src.arch import GFGCN, GFGCNLayer, GFGCN_noh_Layer, GFGCN_Spows

# SEED = 0
SEED = 15
PATH = 'results/diff_filters/'
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
print(device)

torch.manual_seed(SEED)

cuda:1


IDEAS:
- nº capas?
- Orden del filtro?
- normalizaciones del filtro?
- Inicializar BIEN h0?? --> parece que funciona
- Un optimizador o dos? Varias iteraciones?

In [2]:
# Dataset must be from DGL
dataset_name = 'CoraGraphDataset'

A, feat, labels, n_class, masks = utils.get_data_dgl(dataset_name, dev=device,
                                                     verb=True)
N = A.shape[0]

Dataset: CoraGraphDataset
Number of nodes: 2708
Number of features: 1433
Shape of signals: torch.Size([2708, 1433])
Number of classes: 7
Norm of A: 102.74239349365234
Max value of A: 1.0
Proportion of validation data: 0.18
Proportion of test data: 0.37


In [3]:
# Common parameters - training
N_RUNS = 25
N_EPOCHS = 200
LR = .01
WD = 5e-4

# Common parameters - GNN
N_LAYERS = 2
K = 2
IN_DIM = feat.shape[1]
OUT_DIM = n_class
HID_DIM = 16
DROPOUT = 0
ACT = nn.ReLU()
LAST_ACT = nn.LogSoftmax(dim=1)
LOSS_FN = nn.NLLLoss()

## Comparing Models

In [4]:
# Auxiliary functions
def compute_S_pows(S, K, device):
    N = S.shape[0]
    S_pows = torch.Tensor(torch.empty(K-1, N, N)).to(device)
    S_pows[0,:,:] = torch.Tensor(S).to(device)
    for k in range(1,K-1):
        S_pows[k,:,:] = S_pows[0,:,:] @ S_pows[k-1,:,:]

    return S_pows

def get_h_weights(arch, n_layers, K):
     h = np.zeros((n_layers, K))
     for i, layer in enumerate(arch.convs):
         h[i,:] = layer.h.cpu().detach().numpy()
     
     return h

In [5]:
EXPS = [
        {'name': 'Kipf'},
        {'name': 'normA-GCNN', 'h0': 1},
        {'name': 'normA-GCNN', 'h0': .01},
        {'name': 'normH-GCNN-1opt'},
        {'name': 'normH-GCNN-Alt', 'epochs': 200, 'epochs_h': 1, 'epochs_W': 1},
        {'name': 'normH-GCNN-Alt', 'epochs': 50, 'epochs_h': 1, 'epochs_W': 5},
        {'name': 'normH-GCNN-Alt', 'epochs': 25, 'epochs_h': 1, 'epochs_W': 10},
        {'name': 'normH-GCNN-Alt', 'epochs': 10, 'epochs_h': 5, 'epochs_W': 25},
        {'name': 'Kipf-GSO', 'norm': False, 'epochs': 150, 'epochs_h': 1, 'epochs_W': 1},
        {'name': 'Kipf-GSO', 'norm': True, 'epochs': 150, 'epochs_h': 1, 'epochs_W': 1},
]

In [6]:
losses = []
accs = []
h_weights = np.zeros((N_LAYERS, K, len(EXPS), N_RUNS))
best_accs = np.zeros((len(EXPS), N_RUNS))
ellapsed_times = np.zeros((len(EXPS), N_RUNS))
for i in range(N_RUNS):
    print(f'- RUN: {i+1}')
    losses.append([])
    accs.append([])
    for j, exp in enumerate(EXPS):
        t_i = time.time()
        if exp['name'] == 'Kipf':
            arch = GCNN_2L(IN_DIM, HID_DIM, OUT_DIM, act=ACT, last_act=LAST_ACT,
                           dropout=DROPOUT)
            S = dgl.from_networkx(nx.from_numpy_array(A)).add_self_loop().to(device)
            model = NodeClassModel(arch, S, masks, LOSS_FN, device=device)

        if exp['name'] == 'Kipf-GSO':
            arch = GFGCN_Spows(IN_DIM, HID_DIM, OUT_DIM, N_LAYERS, K, act=ACT, last_act=LAST_ACT,
                               dropout=DROPOUT, norm=exp['norm'], dev=device)
            S = torch.Tensor(normalize_gso(A + np.eye(N), 'both')).to(device)
            model = GF_NodeClassModel(arch, S, K, masks, LOSS_FN, device=device)
            
        elif exp['name'] == 'normA-GCNN':
            arch = GFGCN(IN_DIM, HID_DIM, OUT_DIM, N_LAYERS, K, act=ACT, last_act=LAST_ACT,
                         dropout=DROPOUT, diff_layer=GFGCNLayer, init_h0=exp['h0'])
            S = torch.Tensor(normalize_gso(A, 'both')).to(device)
            model = GF_NodeClassModel(arch, S, K, masks, LOSS_FN, device=device)

        elif exp['name'] == 'normH-GCNN-1opt':
            arch = GFGCN_Spows(IN_DIM, HID_DIM, OUT_DIM, N_LAYERS, K, act=ACT, last_act=LAST_ACT,
                               dropout=DROPOUT, norm=True, dev=device)
            S = compute_S_pows(A, K, device)
            model = NodeClassModel(arch, S, masks, LOSS_FN, device=device)

        elif exp['name'] == 'normH-GCNN-Alt':
            arch = GFGCN_Spows(IN_DIM, HID_DIM, OUT_DIM, N_LAYERS, K, act=ACT, last_act=LAST_ACT,
                               dropout=DROPOUT, norm=True, dev=device)
            S = torch.Tensor(A).to(device)
            model = GF_NodeClassModel(arch, S, K, masks, LOSS_FN, device=device)


        if exp['name'] in ['normH-GCNN-Alt', 'Kipf-GSO']:
            loss, acc = model.train(feat, labels, exp['epochs'], LR, WD, epochs_h=exp['epochs_h'],
                                    epochs_W=exp['epochs_W'])
        else:
            loss, acc = model.train(feat, labels, N_EPOCHS, LR, WD)

        ellapsed_t = (time.time()-t_i)/60
        best_accs[j,i] = np.max(acc["test"])
        losses[i].append(loss)
        accs[i].append(acc)
        ellapsed_times[j,i] = ellapsed_t

        if exp['name'] != 'Kipf':
            h_weights[:,:,j,i] = get_h_weights(arch, N_LAYERS, K)

        print(f'\t{exp["name"]}: acc = {best_accs[j,i]:.3f} - time = {ellapsed_t:.2f} mins')
        

- RUN: 1
	Kipf: acc = 0.814 - time = 0.10 mins
	normA-GCNN: acc = 0.780 - time = 0.05 mins
	normA-GCNN: acc = 0.804 - time = 0.09 mins
	normH-GCNN-1opt: acc = 0.768 - time = 0.36 mins
	normH-GCNN-Alt: acc = 0.788 - time = 0.82 mins
	normH-GCNN-Alt: acc = 0.818 - time = 0.10 mins
	normH-GCNN-Alt: acc = 0.814 - time = 0.07 mins
	normH-GCNN-Alt: acc = 0.808 - time = 0.19 mins
	Kipf-GSO: acc = 0.760 - time = 0.56 mins
	Kipf-GSO: acc = 0.680 - time = 0.65 mins
- RUN: 2
	Kipf: acc = 0.820 - time = 0.03 mins
	normA-GCNN: acc = 0.775 - time = 0.02 mins
	normA-GCNN: acc = 0.822 - time = 0.02 mins
	normH-GCNN-1opt: acc = 0.773 - time = 0.30 mins
	normH-GCNN-Alt: acc = 0.762 - time = 0.52 mins
	normH-GCNN-Alt: acc = 0.812 - time = 0.62 mins
	normH-GCNN-Alt: acc = 0.813 - time = 0.08 mins
	normH-GCNN-Alt: acc = 0.805 - time = 0.07 mins
	Kipf-GSO: acc = 0.773 - time = 0.68 mins
	Kipf-GSO: acc = 0.670 - time = 0.18 mins
- RUN: 3
	Kipf: acc = 0.804 - time = 0.05 mins
	normA-GCNN: acc = 0.788 - time =

In [7]:
# Print results
mean_accs = best_accs.mean(axis=1)
med_accs = np.median(best_accs, axis=1)
std_accs = best_accs.std(axis=1)
mean_t = ellapsed_times.mean(axis=1)
for i, exp in enumerate(EXPS):
    name = exp['name']
    if exp['name'] == 'normA-GCNN':
        name += f', h0: {exp["h0"]}'
    elif exp['name'] == 'normH-GCNN-Alt':
        name += f', {exp["epochs"]}-{exp["epochs_h"]}-{exp["epochs_W"]}'

    print(f'{name}:\n \tmean: {mean_accs[i]:.3f} - std: {std_accs[i]:.4f} - med: {med_accs[i]:.3f} - time: {mean_t[i]:.2f} mins')


Kipf:
 	mean: 0.814 - std: 0.0065 - med: 0.814 - time: 0.04 mins
normA-GCNN, h0: 1:
 	mean: 0.777 - std: 0.0071 - med: 0.777 - time: 0.02 mins
normA-GCNN, h0: 0.01:
 	mean: 0.807 - std: 0.0070 - med: 0.808 - time: 0.03 mins
normH-GCNN-1opt:
 	mean: 0.774 - std: 0.0126 - med: 0.772 - time: 0.28 mins
normH-GCNN-Alt, 200-1-1:
 	mean: 0.772 - std: 0.0183 - med: 0.775 - time: 0.40 mins
normH-GCNN-Alt, 50-1-5:
 	mean: 0.813 - std: 0.0060 - med: 0.812 - time: 0.15 mins
normH-GCNN-Alt, 25-1-10:
 	mean: 0.813 - std: 0.0044 - med: 0.813 - time: 0.12 mins
normH-GCNN-Alt, 10-5-25:
 	mean: 0.811 - std: 0.0051 - med: 0.809 - time: 0.13 mins
Kipf-GSO:
 	mean: 0.764 - std: 0.0076 - med: 0.764 - time: 0.31 mins
Kipf-GSO:
 	mean: 0.679 - std: 0.0123 - med: 0.680 - time: 0.26 mins


## Model with unormalized GF

### Normalized vs unormalized GSO

In [8]:
EXPS = [{'normA': True, 'h0': .01},
        {'normA': False, 'h0': .01},
        {'normA': True, 'h0': 1},
        {'normA': False, 'h0': 1},]

best_accs = np.zeros((len(EXPS), N_RUNS))
ellapsed_times = np.zeros((len(EXPS), N_RUNS))
for i in range(N_RUNS):
    for j, exp in enumerate(EXPS):
        t_i = time.time()
        arch = GFGCN(IN_DIM, HID_DIM, OUT_DIM, N_LAYERS, K, act=ACT, last_act=LAST_ACT,
                     dropout=DROPOUT, init_h0=exp['h0'])
        S = normalize_gso(A, 'both') if exp['normA'] else A
        S = torch.Tensor(S).to(device)
        model = GF_NodeClassModel(arch, S, K, masks, LOSS_FN, device=device)
        _, acc = model.train(feat, labels, N_EPOCHS, LR, WD)

        ellapsed_t = (time.time()-t_i)/60
        best_accs[j,i] = np.max(acc["test"])
        ellapsed_times[j,i] = ellapsed_t

        print(f'- {i+1} NormA {exp["normA"]}, h0 {exp["h0"]}: acc = {best_accs[j,i]:.3f} - time = {ellapsed_t:.2f} mins')


- 1 NormA True, h0 0.01: acc = 0.809 - time = 0.02 mins
- 1 NormA False, h0 0.01: acc = 0.711 - time = 0.01 mins
- 1 NormA True, h0 1: acc = 0.775 - time = 0.02 mins
- 1 NormA False, h0 1: acc = 0.709 - time = 0.02 mins
- 2 NormA True, h0 0.01: acc = 0.805 - time = 0.02 mins
- 2 NormA False, h0 0.01: acc = 0.724 - time = 0.02 mins
- 2 NormA True, h0 1: acc = 0.785 - time = 0.02 mins
- 2 NormA False, h0 1: acc = 0.724 - time = 0.01 mins
- 3 NormA True, h0 0.01: acc = 0.804 - time = 0.02 mins
- 3 NormA False, h0 0.01: acc = 0.696 - time = 0.01 mins
- 3 NormA True, h0 1: acc = 0.767 - time = 0.02 mins
- 3 NormA False, h0 1: acc = 0.729 - time = 0.02 mins
- 4 NormA True, h0 0.01: acc = 0.792 - time = 0.02 mins
- 4 NormA False, h0 0.01: acc = 0.709 - time = 0.01 mins
- 4 NormA True, h0 1: acc = 0.782 - time = 0.02 mins
- 4 NormA False, h0 1: acc = 0.719 - time = 0.02 mins
- 5 NormA True, h0 0.01: acc = 0.809 - time = 0.02 mins
- 5 NormA False, h0 0.01: acc = 0.714 - time = 0.02 mins
- 5 Nor

In [9]:
# Print results
mean_accs = best_accs.mean(axis=1)
std_accs = best_accs.std(axis=1)
mean_t = ellapsed_times.mean(axis=1)
for i, exp in enumerate(EXPS):
    print(f'NormA {exp["normA"]}, h0 {exp["h0"]}: \tacc: {mean_accs[i]:.3f} - std: {std_accs[i]:.4f} - time: {mean_t[i]:.2f} mins')

NormA True, h0 0.01: 	acc: 0.806 - std: 0.0058 - time: 0.02 mins
NormA False, h0 0.01: 	acc: 0.716 - std: 0.0173 - time: 0.01 mins
NormA True, h0 1: 	acc: 0.777 - std: 0.0066 - time: 0.02 mins
NormA False, h0 1: 	acc: 0.720 - std: 0.0179 - time: 0.02 mins


### Initialization of h0

In [10]:
h0s = [.001, .005, .01, .05, .1, .5]

best_accs = np.zeros((len(h0s), N_RUNS))
ellapsed_times = np.zeros((len(h0s), N_RUNS))
for i in range(N_RUNS):
    for j, h0 in enumerate(h0s):
        t_i = time.time()
        arch = GFGCN(IN_DIM, HID_DIM, OUT_DIM, N_LAYERS, K, act=ACT, last_act=LAST_ACT,
                     dropout=DROPOUT, init_h0=h0)

        # ASSUMING NORM A IS BETTER
        S = torch.Tensor(normalize_gso(A, 'both')).to(device)
        model = GF_NodeClassModel(arch, S, K, masks, LOSS_FN, device=device)
        _, acc = model.train(feat, labels, N_EPOCHS, LR, WD)

        ellapsed_t = (time.time()-t_i)/60
        best_accs[j,i] = np.max(acc["test"])
        ellapsed_times[j,i] = ellapsed_t

        print(f'- {i+1} h0 {h0}: acc = {best_accs[j,i]:.3f} - time = {ellapsed_t:.2f} mins')
        

- 1 h0 0.001: acc = 0.810 - time = 0.02 mins
- 1 h0 0.005: acc = 0.799 - time = 0.02 mins
- 1 h0 0.01: acc = 0.796 - time = 0.03 mins
- 1 h0 0.05: acc = 0.806 - time = 0.02 mins
- 1 h0 0.1: acc = 0.802 - time = 0.02 mins
- 1 h0 0.5: acc = 0.787 - time = 0.02 mins
- 2 h0 0.001: acc = 0.801 - time = 0.02 mins
- 2 h0 0.005: acc = 0.812 - time = 0.02 mins
- 2 h0 0.01: acc = 0.798 - time = 0.02 mins
- 2 h0 0.05: acc = 0.799 - time = 0.02 mins
- 2 h0 0.1: acc = 0.815 - time = 0.02 mins
- 2 h0 0.5: acc = 0.793 - time = 0.02 mins
- 3 h0 0.001: acc = 0.795 - time = 0.02 mins
- 3 h0 0.005: acc = 0.815 - time = 0.02 mins
- 3 h0 0.01: acc = 0.817 - time = 0.02 mins
- 3 h0 0.05: acc = 0.793 - time = 0.02 mins
- 3 h0 0.1: acc = 0.795 - time = 0.02 mins
- 3 h0 0.5: acc = 0.796 - time = 0.02 mins
- 4 h0 0.001: acc = 0.803 - time = 0.02 mins
- 4 h0 0.005: acc = 0.801 - time = 0.02 mins
- 4 h0 0.01: acc = 0.803 - time = 0.02 mins
- 4 h0 0.05: acc = 0.801 - time = 0.02 mins
- 4 h0 0.1: acc = 0.797 - time

In [11]:
# Print results
mean_accs = best_accs.mean(axis=1)
std_accs = best_accs.std(axis=1)
mean_t = ellapsed_times.mean(axis=1)
for i, h0 in enumerate(h0s):
    print(f'h0 {h0}: \tacc: {mean_accs[i]:.3f} - std: {std_accs[i]:.4f} - time: {mean_t[i]:.2f} mins')

h0 0.001: 	acc: 0.807 - std: 0.0061 - time: 0.02 mins
h0 0.005: 	acc: 0.802 - std: 0.0056 - time: 0.02 mins
h0 0.01: 	acc: 0.806 - std: 0.0071 - time: 0.02 mins
h0 0.05: 	acc: 0.806 - std: 0.0071 - time: 0.02 mins
h0 0.1: 	acc: 0.804 - std: 0.0070 - time: 0.03 mins
h0 0.5: 	acc: 0.791 - std: 0.0048 - time: 0.02 mins


### Joint vs separate GD update for h and W

In [12]:
EXPS = [{'model': 'NodeClassModel', 'epochs': 200},
        {'model': 'NodeClassModel', 'epochs': 300},
        {'model': 'GF_NodeClassModel', 'epochs': 300, 'epochs_h': 1, 'epochs_W': 1},
        {'model': 'GF_NodeClassModel', 'epochs': 300, 'epochs_h': 0, 'epochs_W': 1},
        {'model': 'GF_NodeClassModel', 'epochs': 50, 'epochs_h': 1, 'epochs_W': 5},
        {'model': 'GF_NodeClassModel', 'epochs': 50, 'epochs_h': 5, 'epochs_W': 1},
        {'model': 'GF_NodeClassModel', 'epochs': 50, 'epochs_h': 5, 'epochs_W': 5},
        {'model': 'GF_NodeClassModel', 'epochs': 50, 'epochs_h': 1, 'epochs_W': 10},
        {'model': 'GF_NodeClassModel', 'epochs': 50, 'epochs_h': 1, 'epochs_W': 25},
        {'model': 'GF_NodeClassModel', 'epochs': 50, 'epochs_h': 5, 'epochs_W': 10},
        {'model': 'GF_NodeClassModel', 'epochs': 50, 'epochs_h': 5, 'epochs_W': 25},]
best_init_h0 = .01

best_accs = np.zeros((len(EXPS), N_RUNS))
ellapsed_times = np.zeros((len(EXPS), N_RUNS))
for i in range(N_RUNS):
    for j, exp in enumerate(EXPS):
        t_i = time.time()
        arch = GFGCN(IN_DIM, HID_DIM, OUT_DIM, N_LAYERS, K, act=ACT, last_act=LAST_ACT,
                     dropout=DROPOUT, init_h0=best_init_h0)

        # ASSUMING NORM A IS BETTER
        S = torch.Tensor(normalize_gso(A, 'both')).to(device)

        if exp['model'] == 'NodeClassModel':
            model = NodeClassModel(arch, S, masks, LOSS_FN, device=device)
            _, acc = model.train(feat, labels, exp['epochs'], LR, WD)

        elif exp['model'] == 'GF_NodeClassModel':
            model = GF_NodeClassModel(arch, S, K, masks, LOSS_FN, device=device)
            _, acc = model.train(feat, labels, exp['epochs'], LR, WD, 
                                    epochs_h=exp['epochs_h'], epochs_W=exp['epochs_W'])
        else:
            raise Exception('Unknown model')
        

        ellapsed_t = (time.time()-t_i)/60
        best_accs[j,i] = np.max(acc["test"])
        ellapsed_times[j,i] = ellapsed_t

        print(f'- {i+1}. exp: {j+1}: acc = {best_accs[j,i]:.3f} - time = {ellapsed_t:.2f} mins')


- 1. exp: 1: acc = 0.817 - time = 0.01 mins
- 1. exp: 2: acc = 0.809 - time = 0.02 mins
- 1. exp: 3: acc = 0.804 - time = 0.03 mins
- 1. exp: 4: acc = 0.813 - time = 0.03 mins
- 1. exp: 5: acc = 0.786 - time = 0.02 mins
- 1. exp: 6: acc = 0.808 - time = 0.01 mins
- 2. exp: 1: acc = 0.806 - time = 0.01 mins
- 2. exp: 2: acc = 0.810 - time = 0.02 mins
- 2. exp: 3: acc = 0.804 - time = 0.03 mins
- 2. exp: 4: acc = 0.806 - time = 0.02 mins
- 2. exp: 5: acc = 0.756 - time = 0.03 mins
- 2. exp: 6: acc = 0.802 - time = 0.02 mins
- 3. exp: 1: acc = 0.797 - time = 0.01 mins
- 3. exp: 2: acc = 0.807 - time = 0.02 mins
- 3. exp: 3: acc = 0.795 - time = 0.03 mins
- 3. exp: 4: acc = 0.816 - time = 0.02 mins
- 3. exp: 5: acc = 0.744 - time = 0.02 mins
- 3. exp: 6: acc = 0.813 - time = 0.03 mins
- 4. exp: 1: acc = 0.802 - time = 0.01 mins
- 4. exp: 2: acc = 0.818 - time = 0.02 mins
- 4. exp: 3: acc = 0.814 - time = 0.03 mins
- 4. exp: 4: acc = 0.816 - time = 0.02 mins
- 4. exp: 5: acc = 0.757 - time 

In [13]:
# Print results
mean_accs = best_accs.mean(axis=1)
std_accs = best_accs.std(axis=1)
mean_t = ellapsed_times.mean(axis=1)
for i, exp in enumerate(EXPS):
    text = f'{exp["model"]}, {exp["epochs"]}'
    if exp["model"] == 'GF_NodeClassModel':
        text += f'-{exp["epochs_h"]}-{exp["epochs_W"]}:'
    else:
        text += ':\t'
    print(f'{text} \tacc: {mean_accs[i]:.3f} - std: {std_accs[i]:.4f} - time: {mean_t[i]:.2f} mins')





NodeClassModel, 200:	 	acc: 0.809 - std: 0.0084 - time: 0.02 mins
NodeClassModel, 300:	 	acc: 0.809 - std: 0.0058 - time: 0.02 mins
GF_NodeClassModel, 300-1-1: 	acc: 0.807 - std: 0.0067 - time: 0.03 mins
GF_NodeClassModel, 50-1-5: 	acc: 0.814 - std: 0.0070 - time: 0.02 mins
GF_NodeClassModel, 50-5-1: 	acc: 0.763 - std: 0.0130 - time: 0.02 mins
GF_NodeClassModel, 50-5-5: 	acc: 0.804 - std: 0.0058 - time: 0.02 mins


## Alternating model normalizing the filter

- norm H vs not norm H + 1 vs 2 opts
- diff values for the epochs